<a href="https://colab.research.google.com/github/Neel7317/Generative-Models/blob/main/GPT2_extractor_base(fine_tuning).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch import nn
from torch.utils.data import DataLoader, Dataset as TorchDataset
from transformers import GPT2Tokenizer, GPT2Model

# Sample Dataset
data = [
    {"text": "The soccer team clinched a thrilling victory in overtime.", "label": 0},
    {"text": "The cricket championship finals attracted global attention.", "label": 0},
    {"text": "A new record was set in the 100-meter sprint.", "label": 0},
    {"text": "Tennis legends face off in a historic grand slam final.", "label": 0},
    {"text": "The basketball league announces the upcoming season schedule.", "label": 0},
    {"text": "Fans cheer as their favorite football club wins the title.", "label": 0},
    {"text": "Olympic gold medalist announces retirement from the sport.", "label": 0},
    {"text": "The marathon saw record participation this year.", "label": 0},
    {"text": "Baseball's star player hits a remarkable home run.", "label": 0},
    {"text": "The Formula 1 race witnessed a dramatic last-lap overtake.", "label": 0},
    {"text": "New training facilities unveiled for the national hockey team.", "label": 0},
    {"text": "The underdog team surprises everyone with a stunning win.", "label": 0},
    {"text": "The swimming championship showcased extraordinary talent.", "label": 0},
    {"text": "The coach shares strategies ahead of the semi-finals.", "label": 0},
    {"text": "Athletes gear up for the international sports meet.", "label": 0},
    {"text": "The volleyball team secures a crucial win in the tournament.", "label": 0},
    {"text": "The government unveils a new healthcare policy.", "label": 1},
    {"text": "The president addresses the nation on economic reforms.", "label": 1},
    {"text": "The election results spark debates across the country.", "label": 1},
    {"text": "Parliament passes a bill to boost renewable energy.", "label": 1},
    {"text": "Diplomatic talks between the two nations yield progress.", "label": 1},
    {"text": "The mayor announces plans for urban development.", "label": 1},
    {"text": "The opposition criticizes the new tax policies.", "label": 1},
    {"text": "Campaigns heat up as candidates rally for voter support.", "label": 1},
    {"text": "The new law aims to strengthen cybersecurity measures.", "label": 1},
    {"text": "The prime minister visits a neighboring country for trade talks.", "label": 1},
    {"text": "Protests erupt against the controversial policy decision.", "label": 1},
    {"text": "A landmark decision by the court shapes future legislation.", "label": 1},
    {"text": "The minister highlights plans to improve education systems.", "label": 1},
    {"text": "National security is at the forefront of the political agenda.", "label": 1},
    {"text": "The governor outlines disaster relief efforts after the floods.", "label": 1},
    {"text": "A major treaty is signed to address climate change issues.", "label": 1},
    {"text": "Students prepare for university entrance exams.", "label": 2},
    {"text": "The new study techniques promise better learning outcomes.", "label": 2},
    {"text": "The education board announces changes in the curriculum.", "label": 2},
    {"text": "A groundbreaking study explores the effects of online learning.", "label": 2},
    {"text": "A new scholarship program supports underprivileged students.", "label": 2},
    {"text": "Teachers attend workshops on integrating technology in classrooms.", "label": 2},
    {"text": "The library adds hundreds of new academic resources.", "label": 2},
    {"text": "Researchers publish findings on improving memory retention.", "label": 2},
    {"text": "Study groups form to tackle challenging science topics.", "label": 2},
    {"text": "Students collaborate on innovative projects for the science fair.", "label": 2},
    {"text": "The school introduces coding classes for young learners.", "label": 2},
    {"text": "An education conference discusses the future of AI in learning.", "label": 2},
    {"text": "Experts share tips for mastering difficult math concepts.", "label": 2},
    {"text": "The university announces free courses on climate studies.", "label": 2},
    {"text": "A recent survey highlights the need for mental health support in schools.", "label": 2},
    {"text": "The tutoring center reports an increase in attendance.", "label": 2},
    {"text": "Students celebrate as their project wins the national innovation award.", "label": 2},
    {"text": "A new app helps students track their study habits effectively.", "label": 2},
]

# Split the data into training and validation sets
train_texts, val_texts = train_test_split(data, test_size=0.2, random_state=42)
train_dataset = Dataset.from_list(train_texts)
val_dataset = Dataset.from_list(val_texts)

# Tokenizer and Feature Extractor (GPT-2)
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set padding token as EOS

model = GPT2Model.from_pretrained("distilgpt2")

# Extract features using GPT-2
def extract_features(examples):
    inputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # Extract the [CLS] token's representation from the last layer (the output hidden state)
    return {"features": outputs.last_hidden_state[:, 0, :].numpy()}  # Use the first token's embedding (CLS token)

train_dataset = train_dataset.map(extract_features, batched=True)
val_dataset = val_dataset.map(extract_features, batched=True)

# Prepare label encoding
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform([item['label'] for item in train_texts])
val_labels = label_encoder.transform([item['label'] for item in val_texts])

# Define the Neural Network
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return self.softmax(x)

# Define the dataset for PyTorch
class FeatureDataset(TorchDataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.labels[idx], dtype=torch.long)

# Prepare datasets for PyTorch
train_features = [item["features"] for item in train_dataset]
val_features = [item["features"] for item in val_dataset]

train_dataset = FeatureDataset(train_features, train_labels)
val_dataset = FeatureDataset(val_features, val_labels)

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=False)

# Initialize NN model
input_dim = 768  # GPT-2 base model output dimension
hidden_dim = 128
output_dim = 3  # Number of classes

model_nn = SimpleNN(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_nn.parameters(), lr=1e-5)

# Training loop
def train_model(model, train_dataloader, val_dataloader, criterion, optimizer, epochs=3):
    for epoch in range(epochs):
        model.train()
        for batch in train_dataloader:
            features, labels = batch
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
        evaluate_model(model, val_dataloader)

# Evaluation loop
def evaluate_model(model, val_dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_dataloader:
            features, labels = batch
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f"Validation Accuracy: {accuracy * 100:.2f}%")

# Train the model
train_model(model_nn, train_dataloader, val_dataloader, criterion, optimizer)

# Save the model
torch.save(model_nn.state_dict(), "./feature_extractor_nn_model.pth")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Epoch 1/3, Loss: 1.1790244579315186
Validation Accuracy: 50.00%
Epoch 2/3, Loss: 1.202850580215454
Validation Accuracy: 50.00%
Epoch 3/3, Loss: 1.0410583019256592
Validation Accuracy: 50.00%


In [4]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

# Function to evaluate the model and return detailed metrics
def evaluate_model_detailed(model, val_dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in val_dataloader:
            features, labels = batch
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute metrics
    accuracy = accuracy_score(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, target_names=["Sports", "Politics", "Education"])
    confusion = confusion_matrix(all_labels, all_preds)

    return accuracy, report, confusion

# Evaluate the trained model on the validation set
val_accuracy, val_classification_report, val_confusion_matrix = evaluate_model_detailed(model_nn, val_dataloader)

print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print("Classification Report:")
print(val_classification_report)
print("Confusion Matrix:")
print(val_confusion_matrix)


Validation Accuracy: 50.00%
Classification Report:
              precision    recall  f1-score   support

      Sports       0.00      0.00      0.00         1
    Politics       0.50      1.00      0.67         5
   Education       0.00      0.00      0.00         4

    accuracy                           0.50        10
   macro avg       0.17      0.33      0.22        10
weighted avg       0.25      0.50      0.33        10

Confusion Matrix:
[[0 1 0]
 [0 5 0]
 [0 4 0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
